In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from shutil import copyfile
import cv2
from PIL import Image
import torchvision.transforms.functional as F

This part is for moving the file to make into Train/Val datasets.

In [ ]:
NOMASK = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\NMFD\successful" # no mask
PMASK = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\PMFD\successful"  # Properly masked
IPMASK = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\IMFD\successful"       # inproperly masked

In [ ]:
LABELS = {NOMASK: 0, PMASK: 1, IPMASK: 2}

In [ ]:
newTrainNM5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000\train\NMFD"
newTrainIM5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000\train\IMFD"
newTrainPM5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000\train\PMFD"

newTrainNM = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask\train\NMFD"
newTrainIM = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask\train\IMFD"
newTrainPM = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask\train\PMFD"

newValNM5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000\val\NMFD"
newValIM5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000\val\IMFD"
newValPM5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000\val\PMFD"

newValNM = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask\val\NMFD"
newValIM = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask\val\IMFD"
newValPM = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask\val\PMFD"

In [ ]:
label = newTrainPM

In [ ]:
'''To move the file into Train/Val set'''
def moveFile(flag=False):
    trainCount = 0
    valCount = 0
    for dirName, subdirList, fileList in os.walk(label):
        for f in fileList:
            if f.endswith(".png") or f.endswith(".jpg"):
                if trainCount < 5000:
                    path = os.path.join(dirName,f)
                    newPath = os.path.join(newTrainPM5000,f)
                    copyfile(path, newPath)
                    trainCount += 1
                if trainCount == 5000 and valCount < 1000:
                    path = os.path.join(dirName,f)
                    newPath = os.path.join(newValPM5000,f)
                    copyfile(path, newPath)
                    valCount += 1

In [ ]:
'''Data augmentation and normalization for training'''
'''Just normalization for validation'''
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


Mask5000 = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask5000"
MaskAll = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMask"
MaskTwoClass = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\FaceMaskTwoClass"

dataDir = MaskAll
image_datasets = {x: datasets.ImageFolder(os.path.join(dataDir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated



In [ ]:

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
print(inputs.shape)
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])
#imshow(out, title=[x for x in classes])

In [ ]:
class Model:
    
    def __init__(self, model, dataloaders, criterion, optimizer, scheduler, num_epochs):
        self.EPOCHS = num_epochs
        self.dataloader = dataloaders
    #     self.IMG_HEIGHT = IMG_HEIGHT
    #     self.IMG_WIDTH  = IMG_WIDTH
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
    
    
    def train(self):
        since = time.time()

        best_model_wts = copy.deepcopy(self.model.state_dict())
        best_acc = 0.0

        for epoch in range(self.EPOCHS):
            print('Epoch {}/{}'.format(epoch, self.EPOCHS - 1))
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    self.model.train()  # Set model to training mode
                else:
                    self.model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in self.dataloader[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    self.optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = self.model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = self.criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            self.optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    self.scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(self.model.state_dict())

            print()

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        self.model.load_state_dict(best_model_wts)
        return self.model
    
    def predict(self, img):
        with torch.no_grad():
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
            X_pred = torch.Tensor(img).view(-1,self.IMG_HEIGHT, self.IMG_WIDTH)
            X_pred = X_pred/255
            X_pred = X_pred.view(-1, 1, self.IMG_HEIGHT, self.IMG_WIDTH)
            X_pred = X_pred.to(device)        
            outputs = self.model(inputs)
            _, preds = torch.max(outputs, 1)
            prediction = torch.argmax(outputs)
            if prediction == 0:
                print('No mask',outputs[0])
                return  0
            elif prediction == 1:
                print('Properly weared mask',outputs[0])  
                return  1
            else:
                print('Inproperly weared mask',outputs[0])     
                return  2

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model)

In [ ]:
resModel = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
densenet = models.densenet161(pretrained=True)
googlenet = models.googlenet(pretrained=True)

In [ ]:
optimizer = optim.Adam(resModel.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
resModel = models.resnet18(pretrained=True)
num_ftrs = resModel.fc.in_features
resModel.fc = nn.Linear(num_ftrs, 2)
resModel = resModel.to(device)
optimizer = optim.Adam(resModel.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
alexnet = models.alexnet(pretrained=True)
# num_ftrs = alexnet.fc.in_features
# alexnet.fc = nn.Linear(num_ftrs, 3)
alexnet = alexnet.to(device)
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
squeezenet = models.squeezenet1_0(pretrained=True)
# num_ftrs = squeezenet.fc.in_features
# squeezenet.fc = nn.Linear(num_ftrs, 3)
squeezenet = squeezenet.to(device)
optimizer = optim.Adam(squeezenet.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16 = vgg16.to(device)
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
densenet = models.densenet161(pretrained=True)
densenet = densenet.to(device)
optimizer = optim.Adam(densenet.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
inception = models.inception_v3(pretrained=True)
inception = inception.to(device)
optimizer = optim.Adam(inception.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 15

In [ ]:
googlenet = models.googlenet(pretrained=True)
googlenet = googlenet.to(device)
num_ftrs = googlenet.fc.in_features
googlenet.fc = nn.Linear(num_ftrs, 3)
optimizer = optim.Adam(googlenet.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()
epoch = 50

In [ ]:
exampleModel = Model(googlenet, dataloaders, criterion, optimizer ,exp_lr_scheduler, epoch)

In [ ]:
modelPATH = r"C:\My Folder\Data Science\Fall 2020\Deep Learning\Project\GoogleNetWholeModel.pth"
model = torch.load(modelPATH)

In [ ]:
exampleModel = Model(model, dataloaders, criterion, optimizer ,exp_lr_scheduler, epoch)

In [ ]:
model = exampleModel.train()

In [ ]:
def face_detect_crop(img, classifier, scaleFactor, minNeighbors, min_size, max_size):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors, minSize=min_size, maxSize=max_size)
    detected = True
    if len(features) == 0:
        x = 0
        y = 0
        w = gray_img.shape[1]
        h = gray_img.shape[0]
        features = [[x, y, w, h]]
        cropped = False
    coords = []
    for (x, y, w, h) in features:
        coords = [x, y, w, h]
    
    cropped_img = img[y:y+h, x:x+h].copy()
    return coords, cropped_img, detected


def prediction(cropped_img, color, model):    #, model
    transformed_img = preprocessImage(cropped_img)    
    outputs = model(transformed_img.to(device))
    prediction = torch.argmax(outputs)
    
    if prediction == 0: 
        color = color['red']
        text = "Improperly weared mask detected"
        
    elif prediction == 1:
        color = color['green']
        text = "No mask detected"
    else:
        color = color['blue']
        text = "Properly weared mask detected"
        
              
    return color, text


def preprocessImage(img):
    transformer = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    inp = torch.from_numpy(img.transpose((2, 0, 1)))
    inp = F.to_pil_image(inp)
    inp = transformer(inp)
    transformed_img = inp.unsqueeze(0)
    return transformed_img


def draw_boundary(img, coords, color, text):
    for (x, y, w, h) in coords:
        cv2.rectangle(img, (x,y), (x+w, y+h), color,2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
    